In [688]:
import numpy as np

In [689]:
def forward_layer_matrixMult(x, w):
    # both w,x are row vectors
    # x : n * d, w : d * 1
    # N = xw

    return x @ w

In [690]:
def backward_layer_matrixMult(x, w):
    # N = xw
    # dN/dw, dN/dx

    return x, w

In [691]:
def forward_layer_biasAdd(N, b):
    # N : n * 1, b : 1 * 1
    # P = N + b

    # N: n * k, b: k * 1
     
    return N + b

In [692]:
def backward_layer_biasAdd(N, b):
    # P = N + b (n * 1)
    # dP/dN = Identity (n * n)
    # dP/db = 1 (n * 1)

    #return np.identity(N.shape[0]), np.ones((N.shape[0], 1))
    return np.identity(N.shape[0]), np.identity(b.shape[0])

In [693]:
def forward_layer_sigmoid(P):
    # Q = 1 / 1 + e^-P_i
    
    return 1 / (1 + np.exp(-P))

In [694]:
def backward_layer_sigmoid(P):
    # dQ_i/dP_i = Q_i * (1 - Q_i)

    Q = forward_layer_sigmoid(P)
    Q1 = Q * (1 - Q)
    
    return np.diag(Q1)

In [695]:
def forward_layer_softMax(Q):
    denom = np.sum(np.exp(Q))
    return np.exp(Q) / denom

In [696]:
def backward_layer_softMax(P):
    # dQ/dP = formula written in copy

    sum_ek = np.sum(np.exp(P))
    n = P.shape[0]

    dQ = np.zeros(n, n)

    for i in range(n):
        for j in range(n):
            if i == j :
                dQ[i][j] = sum_ek
            
            dQ[i][j] -= np.exp(P[i])
            dQ[i][j] *= np.exp(P[j])
            dQ[i][j] /= sum_ek * sum_ek

    return dQ

In [697]:
def forward_layer_meanSqrLoss(P,y):
    # P : n * 1, y : n * 1
    # L = 1/n * sum((P - y) ^ 2)    : (1 * 1)
    
    return np.sum((P - y)**2)
    # k = P - y
    # res = k.T @ k

    # return res / 2

In [698]:
def backward_layer_meanSqrLoss(P, y):
    # L = 1/n * sum((P - y) ^ 2)    : (1 * 1)
    # dL/dP = 2 * (P - y) : (n * 1)

    return 2 * (P - y)

In [699]:
def forward_layer_crossEntropy(P,y):
    #y can be multiclass or 
    #P,y both are matrix of same shape
        # returns loss_vector

    loss = np.log((P ** y))
    return -np.sum(loss) 

In [700]:
def backward_layer_crossEntropy(Q,y):
    # dl/dp[i][j] = Q[i][j] - y[i][j]
    return Q - y

### Q2 Boston Dataset

In [701]:
import pandas as pd
import numpy as np
import sklearn.datasets as sk

In [702]:
dictt = sk.fetch_california_housing()
data = dictt.data
target = dictt.target
data.shape

(20640, 8)

In [703]:
from sklearn.preprocessing import Normalizer

In [704]:
#Normalising dataset

transformer = Normalizer().fit(data)

# mean_vals = np.mean(data, axis=0)
# std_devs = np.std(data, axis=0)
# epsilon = 1e-8
# data = (data - mean_vals) / (std_devs + epsilon)

In [705]:
data = transformer.transform(data)

In [706]:
valid = 20000
test_x = data[valid+1:]
test_y = target[valid+1:]
x = data[0:valid]
y = target[0:valid]

In [707]:
def forwardProp(x,w,b,y):
    N = forward_layer_matrixMult(x,w)
    P = forward_layer_biasAdd(N,b)
    L = forward_layer_meanSqrLoss(P,y)
    print("Error:", L/y.shape[0])
    return N,P

In [708]:
def backwardProp(x,w,b,N,P,y):
    dL_dP = backward_layer_meanSqrLoss(P,y)
    dP_dN,dP_dB = backward_layer_biasAdd(N,b)
    dN_dw,dN_dx = backward_layer_matrixMult(x,w)
    dl_dw = dN_dw.T @ dP_dN @ dL_dP
    dl_db =  dP_dB.T @ dL_dP
    return dl_dw,dl_db

In [709]:
def stochastic_gd(x,w,b,y,alpha,epoch):
    for j in range(epoch):
        i = np.random.randint(y.shape[0])
        x_ar = x[i].reshape(1,-1)
        y_ar = np.array([y[i]])
        N,P = forwardProp(x_ar,w,b,y_ar)
        del_w=x_ar.T @ (2*(y_ar-P))
        del_b=2*(y[i]-P)
        #del_w,del_b = backwardProp(x_ar,w,b,N,P,y_ar)
        # print("del w, del b : ", del_w, del_b)
        w = w - (del_w * alpha)
        b = b - (del_b * alpha)
    return w, b

In [710]:
w = np.random.rand(len(data[0]),1)
b = np.random.rand(1,1)
alpha = 1e-8
epoch = 50000
w_ = w

In [711]:
w,b = stochastic_gd(x,w,b,y,alpha,epoch)

Error: 0.05187795280393564
Error: 0.14150501574318378
Error: 3.379499627370918
Error: 0.03866915619833534
Error: 0.5453040649524955
Error: 4.407753780980757
Error: 0.21680300814931605
Error: 0.2178809070549596
Error: 3.937380787360381
Error: 0.27952189614759787
Error: 0.18476254719980528
Error: 4.996965563588755
Error: 0.05560843607052034
Error: 3.367383112329461
Error: 0.3078920062286305
Error: 0.34469588663367307
Error: 2.019503814481626
Error: 0.06963019949132161
Error: 0.2809952494430462
Error: 0.09930983737003564
Error: 0.6397036808747403
Error: 1.7310349563205176
Error: 0.8475028072792196
Error: 2.088373328943524
Error: 0.8759735129662726
Error: 0.2230339368840214
Error: 0.0043665795748786375
Error: 0.45059879754093135
Error: 13.617729319518016
Error: 0.0004886450588312918
Error: 0.06113036675088172
Error: 0.17126233352453438
Error: 0.6640842403170741
Error: 0.008857623752097239
Error: 0.5181613796454052
Error: 0.14967694361713746
Error: 0.33850441327455755
Error: 0.0085020242487

Error: 0.06652682257344164
Error: 0.056041601444018006
Error: 0.5198837373826808
Error: 0.03180081290381199
Error: 0.3196158932154583
Error: 0.5302816347675979
Error: 3.9354474913191724
Error: 0.0796732964017674
Error: 0.00046568375647571003
Error: 0.046068672337704744
Error: 0.7727557786709157
Error: 14.020496288482654
Error: 0.28586611585707805
Error: 5.300150122620701
Error: 2.072883916270205
Error: 0.20294384409268382
Error: 1.3052318973695582e-05
Error: 1.0939662368509457
Error: 0.21801666478668574
Error: 0.3429440693436398
Error: 0.07442426122362646
Error: 0.049985366067466924
Error: 0.03033029709286287
Error: 3.754385083026919
Error: 0.09031270033924338
Error: 0.14884368622103822
Error: 8.167272008512228
Error: 1.7654981963857888
Error: 0.5093706174772711
Error: 4.661945447294007
Error: 0.016568444186452654
Error: 0.8544082012140891
Error: 6.748428971978461
Error: 0.03087674767601039
Error: 2.78547082614256
Error: 1.1341419369316064
Error: 0.8084247748746101
Error: 4.50929688678

In [712]:
w

array([[0.98150402],
       [0.1001696 ],
       [0.47502045],
       [0.1705074 ],
       [0.56383035],
       [0.59414716],
       [0.22572401],
       [0.65017815]])

In [713]:
N,P = forwardProp(test_x,w,b,test_y)

Error: 963.6556930535909


### Q3 IRIS

In [714]:
d = sk.load_iris()
x = d.data
y = d.target

In [715]:
x.shape

(150, 4)

In [716]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [717]:
def oneHot_encoding(categories ,labels):
    num_samples = len(labels)
    num_categories = len(categories)
    one_hot = np.zeros((num_samples, num_categories))

    # Perform one-hot encoding
    for i, label in enumerate(labels):
        index = categories.index(label)
        one_hot[i, index] = 1
    return one_hot

In [718]:
y_encoded = oneHot_encoding([0,1,2],y)
y_encoded

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

In [719]:
def forwardProp(x,w,b,y):
    N = forward_layer_matrixMult(x,w)
    P = forward_layer_biasAdd(N,b)
    Q = forward_layer_softMax(P)
    L = forward_layer_crossEntropy(Q,y)
    print("Error:", L)
    return Q

In [720]:
w = np.random.rand(x.shape[1],y_encoded.shape[1])
b = np.random.rand(1,y_encoded.shape[1])
w

array([[0.98430381, 0.56452902, 0.34752878],
       [0.32320279, 0.20252699, 0.7946703 ],
       [0.51409095, 0.5964966 , 0.39084084],
       [0.24636494, 0.98268593, 0.23334421]])

In [721]:
def backwardProp(x, Q, y):
    dL_dP = backward_layer_crossEntropy(Q, y)
    dL_dw = x.T @ dL_dP
    dL_db = np.sum(Q-y,axis = 0)
    return dL_dw, dL_db

In [722]:
def gd(x,w,b,y,eta):
    Q = forwardProp(x,w,b,y)
    dL_dw,dL_db = backwardProp(x,Q,y)
    w = w - (eta * dL_dw)
    b = b - (eta * dL_db)
    return w,b

In [723]:
epoch = 10000
eta = 1e-7
for i in range(epoch):
    w,b = gd(x,w,b,y_encoded,eta)

Error: 1250.6784113090341
Error: 1250.6752688213878
Error: 1250.672126861414
Error: 1250.6689854291474
Error: 1250.6658445246228
Error: 1250.6627041478746
Error: 1250.6595642989373
Error: 1250.656424977846
Error: 1250.653286184635
Error: 1250.6501479193385
Error: 1250.6470101819918
Error: 1250.6438729726292
Error: 1250.640736291285
Error: 1250.6376001379945
Error: 1250.6344645127915
Error: 1250.6313294157112
Error: 1250.6281948467877
Error: 1250.6250608060564
Error: 1250.6219272935512
Error: 1250.618794309307
Error: 1250.6156618533585
Error: 1250.6125299257396
Error: 1250.609398526486
Error: 1250.6062676556319
Error: 1250.6031373132116
Error: 1250.6000074992598
Error: 1250.5968782138116
Error: 1250.5937494569007
Error: 1250.5906212285627
Error: 1250.5874935288316
Error: 1250.5843663577423
Error: 1250.5812397153293
Error: 1250.5781136016271
Error: 1250.5749880166709
Error: 1250.5718629604946
Error: 1250.5687384331331
Error: 1250.5656144346212
Error: 1250.5624909649932
Error: 1250.559368

Error: 1249.82523947505
Error: 1249.822245015357
Error: 1249.8192510929848
Error: 1249.8162577079695
Error: 1249.8132648603444
Error: 1249.8102725501453
Error: 1249.807280777407
Error: 1249.8042895421636
Error: 1249.8012988444502
Error: 1249.798308684302
Error: 1249.795319061754
Error: 1249.7923299768402
Error: 1249.7893414295959
Error: 1249.7863534200558
Error: 1249.7833659482549
Error: 1249.780379014228
Error: 1249.7773926180098
Error: 1249.774406759635
Error: 1249.7714214391385
Error: 1249.7684366565554
Error: 1249.7654524119203
Error: 1249.7624687052676
Error: 1249.7594855366333
Error: 1249.756502906051
Error: 1249.753520813556
Error: 1249.7505392591831
Error: 1249.7475582429674
Error: 1249.7445777649434
Error: 1249.741597825146
Error: 1249.7386184236102
Error: 1249.7356395603701
Error: 1249.7326612354618
Error: 1249.7296834489193
Error: 1249.7267062007777
Error: 1249.7237294910715
Error: 1249.720753319836
Error: 1249.7177776871051
Error: 1249.714802592915
Error: 1249.7118280372997

In [724]:
w

array([[1.22993757, 0.85920593, 0.67664372],
       [0.49254171, 0.34008923, 0.94323953],
       [0.58333264, 0.80771766, 0.66820849],
       [0.25734715, 1.04836151, 0.33456348]])

In [725]:
def predict(x,y_encoded,y,w,b):
    Q = forwardProp(x,w,b,y_encoded)
    return 1-np.count_nonzero(y - np.argmax(Q,axis =1))/y.shape[0]

In [726]:
predict(x,y_encoded,y,w,b)

Error: 1251.2946520543883


0.33333333333333337